In [1]:
import os.path as osp
from utils import loadData
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import awkward as ak
import torch
from torch_geometric.data import Dataset, download_url, InMemoryDataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

ModuleNotFoundError: No module named 'torch_geometric'